In [0]:
# Create a delta table for the UPI Txns

spark.sql("""
create table if not exists cdc_feed_project_catalog.default.raw_upi_txns_v1 (
    transaction_id STRING,
    upi_id STRING,
    merchant_id STRING,
    transaction_amount DOUBLE,
    transaction_timestamp TIMESTAMP,
    transaction_status sTRING
)
using delta
TBLPROPERTIES('delta.enableChangeDataFeed' = 'true')
""")

print('created cdc table raw_upi_txns_v1 with CDC enabled')

In [0]:
from delta.tables import DeltaTable

mock_batches = [
    # Batch 1: Insert new transactions
    spark.createDataFrame([
        ("T001", "upi1@bank", "M001", 500.0, "2024-12-21 10:00:00", "initiated"),
        ("T002", "upi2@bank", "M002", 1000.0, "2024-12-21 10:05:00", "initiated"),
        ("T003", "upi3@bank", "M003", 1500.0, "2024-12-21 10:10:00", "initiated"),
    ], ["transaction_id", "upi_id", "merchant_id", "transaction_amount", "transaction_timestamp", "transaction_status"]),

    # Batch 2: Update and insert transactions
    spark.createDataFrame([
        ("T001", "upi1@bank", "M001", 500.0, "2024-12-21 10:15:00", "completed"),  # Update transaction
        ("T002", "upi2@bank", "M002", 1000.0, "2024-12-21 10:20:00", "failed"),    # Update transaction
        ("T004", "upi4@bank", "M004", 2000.0, "2024-12-21 10:25:00", "initiated"), # New transaction
    ], ["transaction_id", "upi_id", "merchant_id", "transaction_amount", "transaction_timestamp", "transaction_status"]),

    # Batch 3: Handle refunds and updates
    spark.createDataFrame([
        ("T001", "upi1@bank", "M001", 500.0, "2024-12-21 10:30:00", "refunded"),  # Refund issued
        ("T003", "upi3@bank", "M003", 1500.0, "2024-12-21 10:35:00", "completed"), # Completed transaction
    ], ["transaction_id", "upi_id", "merchant_id", "transaction_amount", "transaction_timestamp", "transaction_status"]),
]

def merge_to_deltaTable(delta_table_name, batch_df):
    delta_table = DeltaTable.forName(spark, delta_table_name)

    delta_table.alias("target").merge(
        batch_df.alias("source"),
        "target.transaction_id = source.transaction_id"
    ).whenMatchedUpdate(set={
        "upi_id": "source.upi_id",
        "merchant_id": "source.merchant_id",
        "transaction_amount": "source.transaction_amount",
        "transaction_timestamp": "source.transaction_timestamp",
        "transaction_status": "source.transaction_status"
    }).whenNotMatchedInsertAll().execute()

merge_to_deltaTable("cdc_feed_project_catalog.default.raw_upi_txns_v1", mock_batches[2])
